In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import cv2

In [2]:
# control points in image coordinates: P1(305, 440), P2(672, 430), P3(480, 704), P4(971,630)
# destination points in 'world coordinates': P1(0, 0), P2(449, 0), P3(0, 465), P4(449, 465)
def calculate_projective_transformation(control_points, destination_points):
    """ calculates the homography for given control and destination points.
    
    - construct a matrix containing both the control and destination points 
    - calculate the svd to find the homography
    
    """
    affine_trans = []
    
    if not len(control_points) == len(destination_points):
        raise Exception('Number of control points must match number of destination points')
    
    for index in range(0, len(control_points)):
        x_control, y_control = control_points[index]
        x_dest, y_dest = destination_points[index]
        
        matrix_entry_1 = [x_control, y_control, 1, 0, 0, 0, -x_dest*x_control, -x_dest*y_control, -x_dest]
        matrix_entry_2 = [0, 0, 0, x_control, y_control, 1, -y_dest*x_control, -y_dest*y_control, -y_dest]
        
        affine_trans.append(matrix_entry_1)
        affine_trans.append(matrix_entry_2)
    
    affine_trans = np.array(affine_trans)    
    
    u, s, v = np.linalg.svd(affine_trans)
    homography = v[-1, :]
    homography = homography.reshape(3, 3)
    homography = homography / homography[2, 2] 
    
    return homography


def apply_projective_transformation(image_path, control_points, destination_points):
    """
    
    """
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    homography = calculate_projective_transformation(control_points, destination_points)
    
    for point in destination_points:
        point_homogenous = np.append(np.array(point), 1)
        point_trans = np.dot(homography, point_homogenous)
        point_trans /= point_trans[2]
        print(point_trans)
        

In [3]:
def main():
    control_points = [[305, 440], [672, 430], [480, 704], [971, 630]]
    destination_points = [[0,0], [449, 0], [0, 465], [449, 465]]
    
    apply_projective_transformation('schraegbild_tempelhof.jpg', destination_points, control_points)
    
if __name__ == '__main__':
    main()

[818.42924875 632.98183983   1.        ]
[1.10746447e+03 5.86158819e+02 1.00000000e+00]
[1.28675869e+03 8.13855641e+02 1.00000000e+00]
[1.51025840e+03 6.62560246e+02 1.00000000e+00]
